# Summary

I wanted to look at genome size, GC content, and other statistics from NCBI as they vary over the tree of life. Questions of interest are:
1. How do genome sizes vary, and what are their averages, by clade level?
2. How does GC content vary by clade level?

Since my work has hitherto been focused on soils, I focus on fungi, bacteria, and archaea.

## Data sources
We get genome information from the [NCBI genome reports page](ftp://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/), separately for eukaryota and prokaryota.

We get taxonomy information from two sources--
1. For bacteria and archaea, from the well-annotated [GTDB database](https://data.ace.uq.edu.au/public/gtdb/data/releases/latest/)
2. For fungi, genbank (NCBI) will suffice through the `ete3` package

# Initialization

## Imports

In [2]:
import numpy as np
import pandas as pd

## Function definitions

## Retrieve genome information

In [ ]:
df_genomes_euk = pd.read_csv('ftp://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/eukaryotes.txt', sep='\t').set_index('Assembly Accession')

In [ ]:
df_genomes_prok = pd.read_csv('ftp://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/prokaryotes.txt', sep='\t').set_index('Assembly Accession')

## Retrieve taxonomy information

### GTDB

In [3]:
# for bacteria
df_tax_bacteria = pd.read_csv('https://data.ace.uq.edu.au/public/gtdb/data/releases/latest/bac120_taxonomy.tsv', 
                              sep='\t', 
                              names=['accession_id', 'taxstring'])

KeyboardInterrupt: 

In [ ]:
for tl in _TAXONOMIC_LEVEL_DICT.values():
    df_tax_bacteria.loc[:, tl] = df_tax_bacteria['taxstring'].apply(lambda x: get_taxonomic_level(x, tl))
# keep only unique species
df_tax_bacteria = df_tax_bacteria.drop_duplicates(subset=['species'])
# clean up index-- should chnage RS_GCF_900045825.1 to GCF_900045825.1, or GB_GCA_002347035.1 to GCA_002347035.1, a legitimate RefSeq accession ID
df_tax_bacteria.loc[:, 'accession_id'] = df_tax_bacteria['accession_id'].apply(lambda x: '_'.join(x.split('_')[1:]))
df_tax_bacteria = df_tax_bacteria.set_index('accession_id')

In [4]:
# for archaea
df_tax_archaea = pd.read_csv('https://data.ace.uq.edu.au/public/gtdb/data/releases/latest/ar122_taxonomy.tsv', 
                             sep='\t', 
                             names=['accession_id', 'taxstring'])

In [ ]:
for tl in _TAXONOMIC_LEVEL_DICT.values():
    df_tax_archaea.loc[:, tl] = df_tax_archaea['taxstring'].apply(lambda x: get_taxonomic_level(x, tl))
# keep only unique species
df_tax_archaea_unique = df_tax_archaea.drop_duplicates(subset=['species'])
# clean up index-- should chnage RS_GCF_900045825.1 to GCF_900045825.1, or GB_GCA_002347035.1 to GCA_002347035.1, a legitimate RefSeq accession ID
df_tax_archaea_unique.loc[:, 'accession_id'] = df_tax_archaea_unique['accession_id'].apply(lambda x: '_'.join(x.split('_')[1:]))
df_tax_archaea_unique = df_tax_archaea_unique.set_index('accession_id')

## NCBI

To get this data:

Run ncbi-genome-download to get fungi from genbank and get their accession/taxid information:
ncbi-genome-download --format assembly-report --section genbank fungi
This will create a set of folders of the form

..../genbank/fungi/<accession_id>/<accession_id>_<some_alternate_accession_id>_assembly_report.txt
e.g.

genbank/fungi/GCA_003719405.1:
GCA_003719405.1_ASM371940v1_assembly_report.txt

genbank/fungi/GCA_003719415.1:
GCA_003719415.1_ASM371941v1_assembly_report.txt

genbank/fungi/GCA_003719635.1:
GCA_003719635.1_ASM371963v1_assembly_report.txt

genbank/fungi/GCA_003719665.1:
GCA_003719665.1_ASM371966v1_assembly_report.txt
The starting content of these assembly reports looks something like:

```
# Assembly name:  ASM371966v1
# Organism name:  Saccharomyces cerevisiae (baker's yeast)
# Infraspecific name:  strain=X55
# Taxid:          4932
# BioSample:      SAMN10330870
# BioProject:     PRJNA498704
# Submitter:      Okla. Medical Research Foundation
# Date:           2018-11-10
# Assembly type:  haploid
# Release type:   major
# Assembly level: Chromosome
# Genome representation: full
# Assembly method: BBMap v. 38.26
# Expected final version: no
# Reference guided assembly: GCF_000146045.2
# Genome coverage: 250X
# Sequencing technology: Illumina MiSeq
# GenBank assembly accession: GCA_003719665.1
Get the tax IDs for each of the accession IDs/tax IDs gotten from the assembly reports in the line above (see get_taxid_accession_id_from_assembly_report function)
Parse this into a dictionary (see fungal_taxa below) that has the associated full taxstring (including all lineages, derived from ete3.NCBITaxa

# Analysis

## Genome size variation by clade

In [ ]:
df_genomes_euk = df_genomes_euk.loc[[x for x in df_taxonomy.index if x in df_genomes_euk.index]]
df_genomes_euk = df_genomes_euk.merge(df_taxonomy, left_index=True, right_index=True, how='inner')
# drop species duplicates, which will throw stats off-- want to look at species-level variability, not strain-level
df_genomes_euk = df_genomes_euk.drop_duplicates(subset=['species'])
df_genomes_euk = df_genomes_euk.astype({'Size (Mb)': 'float'})#, 'GC%': 'float'})
df_genomes_euk.groupby(['phylum', 'class']).agg({'Size (Mb)': {'mean': np.mean, 'cv': lambda x: np.std(x)/np.mean(x), 'count': 'count'}})